# Peter results

In [ ]:
import pymongo as pm
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import re

## Connect to db

In [ ]:
client = pm.MongoClient()   #We are using local host
db = client["experiment"]

##  Analysis

In [ ]:
a = db["sessions"].count_documents({})
a

### Finding the correct awnser to the problem

In [ ]:

# full_name = pd.DataFrame()
# single_letter = pd.DataFrame()
# abbreviation = pd.DataFrame()
# semantic = pd.DataFrame()
# syntatic = pd.DataFrame()
full_name = []
single_letter = []
abbreviation = []
semantic = []
syntatic = []
abbreviation_regex = "abbrev.html$"
full_name_regex = "normal.html$"
single_letter_regex = "single.html$"
semantic_regex = "..\.semantic.."
syntatic_regex = "..\.syntactic.."


sessions = db["sessions"]
for session in sessions.find({"done": True}):
    for trial in session["trials"]:
        # print(trial)
        # print((trial["file"]))
        if re.search(pattern=full_name_regex, string=trial["file"]):
            full_name.append(trial)
        elif re.search(pattern=abbreviation_regex, string=trial["file"]):
            abbreviation.append(trial)
        elif re.search(pattern=single_letter_regex, string=trial["file"]):
            single_letter.append(trial)
        if re.search(pattern=semantic_regex, string=trial["file"]):
            semantic.append(trial)
        elif re.search(pattern=syntatic_regex, string=trial["file"]):
            syntatic.append(trial)

### Update trials with indicators for type

In [ ]:
def update_trials_with_indecators(collection_name) -> None:
    abbreviation_regex = "abbrev.html$"
    full_name_regex = "normal.html$"
    single_letter_regex = "single.html$"
    semantic_regex = "..\.semantic.."
    syntatic_regex = "..\.syntactic.."


    # sessions = db["sessions"]
    collection = db[collection_name]
    for session in collection.find({"done": True}):
        for trial in session["trials"]:
            variable_type = ""
            challenge_type = ""
            if re.search(pattern=full_name_regex, string=trial["file"]):
                variable_type = "Full variable name"
            elif re.search(pattern=abbreviation_regex, string=trial["file"]):
                variable_type = "Abbreviated variable name"
            elif re.search(pattern=single_letter_regex, string=trial["file"]):
                variable_type = "Single letter variable name"
            
            if re.search(pattern=semantic_regex, string=trial["file"]):
                challenge_type = "Semantic error"
            elif re.search(pattern=syntatic_regex, string=trial["file"]):
                challenge_type = "Syntatic error"

            # {"USER_SESSION_ID": "671f2305ebda49dfa1b12ff179470645", "trials.created": "2025-05-15T08:23:06.814322"}
            filter = {"USER_SESSION_ID": session["USER_SESSION_ID"], "trials.created": trial["created"]}
            operation = {"$set":  {'trials.$.variable_type': str(variable_type)}}
            ops = {"$set": {'trials.$.challenge_type': str(challenge_type)}}
            db.sessions.update_one(filter, operation)
            db.sessions.update_one(filter, ops)

In [ ]:
update_trials_with_indecators("sessions")

In [ ]:
def success_rate(bucket: list) -> [list, list]:
    success = 0
    fail = 0
    for drop in bucket:
        if drop["state"] == "success":
            success += 1
        elif drop["state"] == "failure":
            fail += 1
    return [[success, fail], ["success", "failure"]]

In [ ]:
data, keys = success_rate(single_letter)

# define Seaborn color palette to use
palette_color = sns.color_palette('bright')
0
# plotting data on chart
plt.pie(data, labels=keys, colors=palette_color, autopct='%.0f%%')
plt.title(f"Single letter variable")
# displaying chart
plt.show()

In [ ]:
single_letter_DF = pd.DataFrame(single_letter)
single_letter_DF

In [ ]:
penguins = sns.load_dataset("penguins")
peg_DF = pd.DataFrame(penguins)
peg_DF